In [5]:
import os
os.chdir("/Users/Layne/Desktop/aiqc")

In [6]:
import aiqc
from aiqc import examples

In [7]:
from importlib import reload
aiqc.delete_db(True)
reload(aiqc)
aiqc.create_db()


=> Success - deleted database file at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - created database file for machine learning metrics at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - created the following tables within database:
['algorithm', 'batch', 'dataset', 'featureset', 'file', 'fold', 'foldset', 'hyperparamcombo', 'hyperparamset', 'image', 'job', 'jobset', 'label', 'preprocess', 'result', 'splitset', 'tabular']



---

In [8]:
df = examples.demo_file_to_pandas('iris.tsv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Pipeline for ingesting and splitting up sample data

In [11]:
splitset = aiqc.Pipeline.Tabular.ingest(
    dataFrame_or_filePath = df
    , label_column = 'species'
    , size_test = 0.22
    , size_validation = 0.12
    , fold_count = None
    , bin_count = None
)

### Experiment for tracking models and parameters.

In [12]:
import keras
from keras import metrics
from keras.models import Sequential, load_model
from keras.callbacks import History
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

In [13]:
def function_model_build(**hyperparameters):
    model = Sequential()
    model.add(Dense(13, input_shape=(4,), activation='relu', kernel_initializer='he_uniform', name='fc1')) # first hidden layer
    model.add(Dropout(0.2))
    model.add(Dense(hyperparameters['l2_neuron_count'], activation='relu', kernel_initializer='he_uniform', name='fc2'))
    model.add(Dense(3, activation='softmax', name='output'))

    model.compile(
        loss='categorical_crossentropy'
        , optimizer=hyperparameters['optimizer']
        , metrics=['accuracy']
    )
    return model

In [14]:
def function_model_train(model, samples_train, samples_evaluate, **hyperparameters):
    model.fit(
        samples_train["features"]
        , samples_train["labels"]
        , validation_data = (
            samples_evaluate["features"]
            , samples_evaluate["labels"]
        )
        , verbose = 0
        , batch_size = 3
        , epochs = hyperparameters['epochs']
        , callbacks=[History()]
    )
    return model

In [15]:
hyperparameters = {
    "l2_neuron_count": [9, 13, 18]
    , "optimizer": ["adamax", "adam"]
    , "epochs": [66]
}

In [16]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [17]:
encoder_features = StandardScaler()

In [18]:
encoder_labels = OneHotEncoder(sparse=False)

In [19]:
batch = aiqc.Experiment.stage(
    library = "Keras"
    , analysis_type = "classification_multi"
    , function_model_build = function_model_build
    , function_model_train = function_model_train
    , splitset_id = splitset.id
    , hyperparameters = hyperparameters
    , repeat_count = 1
    , hide_test = False
    , function_model_predict = None
    , function_model_loss = None
    , foldset_id = None
    , encoder_features = encoder_features
    , encoder_labels = encoder_labels
)

In [20]:
batch.run_jobs()

🔮 Training Models 🔮: 100%|██████████████████████████████████████████| 6/6 [00:20<00:00,  3.34s/it]


In [21]:
batch.jobs[0].results[0].metrics

{'test': {'accuracy': 0.9090909090909091,
  'f1': 0.9089026915113873,
  'loss': 0.25114214420318604,
  'precision': 0.9111111111111111,
  'recall': 0.9090909090909091,
  'roc_auc': 0.9917355371900827},
 'validation': {'accuracy': 0.8888888888888888,
  'f1': 0.8857142857142857,
  'loss': 0.23458917438983917,
  'precision': 0.9166666666666666,
  'recall': 0.8888888888888888,
  'roc_auc': 1.0},
 'train': {'accuracy': 0.8888888888888888,
  'f1': 0.8867836257309941,
  'loss': 0.23336029052734375,
  'precision': 0.9067460317460317,
  'recall': 0.8888888888888888,
  'roc_auc': 0.9920416284052648}}